In [4]:
#importing required library
import keras
import os
import glob
import pandas as pd
import numpy as np
import cv2
#%%
#preprocessing the image datas
file_dir1 = 'C:/Users/JAYALAKSHMI/Downloads/input1/input1/*'
file_dir2 = 'C:/Users/JAYALAKSHMI/Downloads/input2-_1_ (1)/input2/*'
files = glob.glob(file_dir1)
print(len(files))
file2 = glob.glob(file_dir2)
print(len(file2))

    
x1 = []
for f1 in files:
  img_dir = os.path.join(f1)
  #print('img_dir',img_dir)
  images = glob.glob(img_dir)
  for img in images:
    img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    #img = cv2.resize(img,dim,ingerpolation = cv2.INTER_AREA)
    img_temp = cv2.resize(img_temp,(500,200))
    img_temo = img_temp/255
    x1.append(img_temp)
import numpy as np
x1 = np.array(x1)
print(x1.shape)

#%%
#file2 = glob.glob(file_dir2)
#print(len(file2))
x2 = []
for f1 in file2:
  img_dir = os.path.join(f1)
  #print('img_dir',img_dir)
  images = glob.glob(img_dir)
  for img in images:
    img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    img_temp = cv2.resize(img_temp,(500,200))
    img_temo = img_temp/255
    x2.append(img_temp)
import numpy as np
x2 = np.array(x2)
print(x2.shape)
#%%
ylabel = 'C:/Users/JAYALAKSHMI/Downloads/ylabel.csv'
y = pd.read_csv(ylabel)
print(y)
print(x1)
#%%
from sklearn.utils import shuffle
x1,x2,y = shuffle(x1,x2,y,random_state=2)

# splitting dataset into train test
x1_train = x1[0:180]
x1_train = x1_train.reshape(180,200,500,1)
x1_val = x1[180:210]
x1_val = x1_val.reshape(30,200,500,1)

x2_train = x2[0:180]
x2_train = x2_train.reshape(180,200,500,1)
x2_val = x2[180:210]
x2_val = x2_val.reshape(30,200,500,1)

x1_test = x1[210 :]
x1_test = x1_test.reshape(40,200,500,1)
x2_test = x2[210 :]
x2_test = x2_test.reshape(40,200,500,1)

y_train = y[0:180]
y_val = y[180:210]
y_test = y[210 :]



Using TensorFlow backend.


250
250
(250, 200, 500)
(250, 200, 500)
     label
0        1
1        1
2        0
3        1
4        1
..     ...
245      0
246      0
247      0
248      1
249      1

[250 rows x 1 columns]
[[[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

 [[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

 [[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

 ...

 [[255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  ...
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]
  [255 255 255 ... 255 255 255]]

In [7]:
#%%
def get_siamese_model(input_shape):
    #input_shape = (200,500,1)
    left_input=Input(input_shape)
    right_input=Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(32,(10,10), activation='relu', input_shape=input_shape,
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling2D())
    model.add(Conv2D(64,(7,7), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())   
    model.add(Conv2D(64,(4,4), activation='relu', 
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    model.add(Conv2D(128,(4,4), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(1024,activation='sigmoid',kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    
    encoded_l=model(left_input)
    encoded_r=model(right_input)
    
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
    L1_distance=L1_layer([encoded_l,encoded_r])
    
    prediction=Dense(1,activation='sigmoid')(L1_distance)
    
    siamese_net1 = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net1



In [8]:
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Lambda, Flatten, Dense

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K
K.set_image_data_format('channels_last')
#from sklearn.utils import shuffle
    
model = get_siamese_model((200,500,1))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print("starting training process")
print("...............................")
#t_start = time.time()
model.fit([x1_train,x2_train],y_train,verbose=1,batch_size=5,validation_data=[[x1_val,x2_val],y_val],epochs = 5,shuffle = True)
scores = model.evaluate([x1_test,x2_test],y_test,verbose=1)
print(scores)
model.save('siamese_net.h5')
#t_stop = (time.time()-t_start())
#print(t_stop)

starting training process
...............................
Train on 180 samples, validate on 30 samples
Epoch 1/5
180/180 [==============================] - ETA: 10:28 - loss: 22.3755 - accuracy: 0.40 - ETA: 8:41 - loss: 21.9424 - accuracy: 0.5000 - ETA: 7:37 - loss: 21.4127 - accuracy: 0.533 - ETA: 6:55 - loss: 20.8736 - accuracy: 0.550 - ETA: 6:27 - loss: 20.4187 - accuracy: 0.480 - ETA: 6:05 - loss: 19.8694 - accuracy: 0.466 - ETA: 5:45 - loss: 19.3404 - accuracy: 0.457 - ETA: 5:27 - loss: 18.8592 - accuracy: 0.425 - ETA: 5:12 - loss: 18.3594 - accuracy: 0.466 - ETA: 4:56 - loss: 17.8826 - accuracy: 0.440 - ETA: 4:42 - loss: 17.4186 - accuracy: 0.436 - ETA: 4:30 - loss: 16.9712 - accuracy: 0.450 - ETA: 4:16 - loss: 16.5510 - accuracy: 0.415 - ETA: 4:04 - loss: 16.1474 - accuracy: 0.400 - ETA: 3:51 - loss: 15.7590 - accuracy: 0.426 - ETA: 3:39 - loss: 15.3918 - accuracy: 0.437 - ETA: 3:27 - loss: 15.0414 - accuracy: 0.458 - ETA: 3:15 - loss: 14.7076 - accuracy: 0.455 - ETA: 3:04 - los

In [53]:
def test_loss():
    in_tensor = tf.placeholder(tf.float32, (None, 3))
    labels = tf.placeholder(tf.int32, None, 1)
    model = Model(in_tensor, labels)
    sess = tf.Session()
    loss = sess.run(model.loss, feed_dict={
    in_tensor:np.ones(1, 3),
    labels:[[1]]})
    assert (loss!= 0)

In [54]:
class GAN:
    def _init_(self, z_vector, true_images):
    # Pretend these are implemented.
        with tf.variable_scope("gen"):
            self.make_geneator(z_vector)
        with tf.variable_scope("des"):
            self.make_descriminator(true_images)
        opt = tf.AdamOptimizer()
        train_descrim = opt.minimize(self.descrim_loss)
        train_gen = opt.minimize(self.gen_loss)

In [55]:
def test_gen_training():
    model = Model
    sess = tf.Session()
    gen_vars = tf.get_collection(tf.GraphKeys.VARIABLES, scope='gen')
    des_vars = tf.get_collection(tf.GraphKeys.VARIABLES, scope='des')
    before_gen = sess.run(gen_vars)
    before_des = sess.run(des_vars)
    # Train the generator.
    sess.run(model.train_gen)
    after_gen = sess.run(gen_vars)
    after_des = sess.run(des_vars)
    # Make sure the generator variables changed.
    for b,a in zip(before_gen, after_gen):
        assert(a != b).any()
  # Make sure descriminator did NOT change.
    for b,a in zip(before_des, after_des):
        assert(a == b).all()

In [22]:
pip install pytest

# TUNING PROCESS

In [5]:
from tensorflow import keras
import kerastuner as kt
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from kerastuner.applications import HyperResNet
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
import tensorflow as tf
from keras import layers

In [6]:
from kerastuner.engine.hyperparameters import HyperParameters as hp
#importing HyperParameters as hp

def get_siamese_model(input_shape):
    input_shape = (200,500,1)
    left_input=Input(input_shape)
    right_input=Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation='relu', input_shape=input_shape,
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(
            Dropout(rate=hp.Float(
                    'dropout_1',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05,
                    
                
            )))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(Conv2D(64,(4,4), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(
        Dropout(
                rate=hp.Float(
                    'dropout2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05,
                )
            )
    )
    model.add(tf.keras.layers.MaxPooling2D())
    for i in range(hp.Int('n_layers',1,4)):
        model.add(Conv2D(hp.Int(f'conv_{i}_units', 
                    min_value=32,
                    max_value=256,
                    step=32),(4,4)))
        model.add(Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(
        Dropout(
            rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05,
            )))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(Conv2D(128,(9,9), activation='relu',
                    kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(tf.keras.layers.MaxPooling2D())
    model.add(Flatten())
    model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
    
    model.add(Dense(1024,activation='sigmoid',kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    
    encoded_l=model(left_input)
    encoded_r=model(right_input)
    
    L1_layer = tf.keras.layers.Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
    L1_distance=L1_layer([encoded_l,encoded_r])
    
    prediction=tf.keras.layers.Dense(1,activation='sigmoid')(L1_distance)
    
    siamese_net1 = Model(inputs=[left_input,right_input],outputs=prediction)
    
    siamese_net1.compile(
     optimizer=tf.keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    return siamese_net1


print("starting training process")
print("************************")
#t_start = time.time()
model.fit([x1_train,x2_train],y_train,verbose=1,batch_size=5,validation_data=[[x1_val,x2_val],y_val],epochs = 5,shuffle = True)
scores = model.evaluate([x1_test,x2_test],y_test,verbose=1)
print(scores)
#model.save('siamese_net.h5')
#t_stop = (time.time()-t_start())
#print(t_stop)



starting training process
************************


NameError: name 'model' is not defined

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from keras.utils import to_categorical


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    #y_binary = to_categorical(180)
    return model

In [13]:
tuner = RandomSearch(
    get_siamese_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    )

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


# Tuner search summary

In [14]:
tuner.search_space_summary()

In [15]:
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel


HYPERBAND_MAX_EPOCHS = 2
MAX_TRIALS = 5
EXECUTION_PER_TRIAL = 1

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='siamese'
)

INFO:tensorflow:Reloading Oracle from existing project hyperband\siamese\oracle.json


# Printing the best model

In [16]:
#best model
tuner.results_summary()